In [ ]:
import os
import civis
import time
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
AFF_SYNC_COLUMNS = ['affiliateid','affiliateein','affiliatepercapitapin','IsChartered','CharterDate','electionmonth','officertermstartmonth','iselectionyearodd','electiontermyears','updatedby','updatedat']
APC_SYNC_COLUMNS = ['AffiliatePerCapitaId','FiscalYearEndMonth','FiscalYearEndDay','PayPerCapitaToAFT','InvoicedByAFT','IncludeAFLCIOPerCapita','AFLCIOAmount','AffiliateBillingFrequencyId','HasOccupationalLiabilityInsurance','FiduciaryBondCoverageID','AccidentInsuranceUnits','ConventionDelegationEligibility','IsAgencyFee','IsStateDues','DeliveryType','GroupNumber','UpdatedBy','UpdatedAt','DeletedAt']
ADIV_SYNC_COLUMNS = ['Action','AffiliateDivisionId','Affiliate','Division','CreatedBy','CreatedAt','UpdatedBy','UpdatedAt','DeletedAt']

YESTERDAY_DATE = (date.today() - timedelta(days = 1)).strftime("%m/%d/%Y")
DATE_TO_RUN = YESTERDAY_DATE

TABLES = ['Affiliate','Division','AffiliateDivision','AffiliatePerCapita','Accounts','StateFederation','AffiliateType','AffiliateDesignation','AffiliateGeoReach']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name']:item['key']})
                self.APP_DICT.update({name:{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_records_updated_at_date(self, knack_object, date):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['mdate']
        aftdbupdate_id = self.APP_DICT[knack_object]['fields']['AFTDBUpdate']

        #Get Id {'field':field_to_match_id, 'operator':'is', 'value':date}, 
        match_filter = {'match':'and', 'rules':[{'field':aftdbupdate_id, 'operator':'is', 'value':0}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url + "&rows_per_page=1000"

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_connection(self,connection_name,field,connection):
        #Convert to IDs
        knack_object_id = self.APP_DICT[connection_name]['obj_id']
        knack_field_id = self.APP_DICT[connection_name]['fields'][field]
        
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  connection
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict:
            return res_json_dict[knack_field_id]
        else:
            return ''


client = KnackAFT()
aff_fields = {}
apc_fields = {}
adiv_fields = {}

#Temp Fixes for naming convention, will address permenantely in API Wrapper 2.0
aff_temp = client.APP_DICT['Affiliate']['fields']
aff_temp.update({
    'affiliateein':aff_temp.pop('Affiliate EIN'),
    'affiliatepercapitapin':aff_temp.pop('Affiliate Percapita Pin'),
    'IsChartered':aff_temp.pop('Is Affiliate Chartered'),
    'CharterDate':aff_temp.pop('Charter Date'),
    'ParentAffiliateID':aff_temp.pop('State Federation'),
    'AffiliateTypeID':aff_temp.pop('Affiliate Type'),
    'AffiliateDesignationID':aff_temp.pop('Affiliate Designation'),
    'AffiliateGeoReachID':aff_temp.pop('Affiliate Geographical Reach'),
    'LocationStateAbr':aff_temp.pop('Location State')
})

for name in AFF_SYNC_COLUMNS:
    aff_fields.update({name:client.APP_DICT['Affiliate']['fields'][name]})
        
for name in APC_SYNC_COLUMNS:
    apc_fields.update({name:client.APP_DICT['AffiliatePerCapita']['fields'][name]})
    
for name in ADIV_SYNC_COLUMNS:
    adiv_fields.update({name:client.APP_DICT['AffiliateDivision']['fields'][name]})

def affiliate_runner():
    output_df = []
    records = client.find_records_updated_at_date('Affiliate', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in aff_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['Affiliate']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['Affiliate']['fields']['mdate']]
            muser = client.get_userid(user_conn)

            checker = record[client.APP_DICT['Affiliate']['fields']['ParentAffiliateID']+'_raw']
            if checker:
                val = client.get_connection('StateFederation','AffiliateId',checker[0]['id'])
                output.update({'ParentAffiliateID':val})
            else:
                output.update({'ParentAffiliateID':''})

            val = client.get_connection('AffiliateType','AffiliateTypeId',record[client.APP_DICT['Affiliate']['fields']['AffiliateTypeID']+'_raw'][0]['id'])
            output.update({'AffiliateTypeID':val})
            
            val = client.get_connection('AffiliateDesignation','AffiliateDesignationId',record[client.APP_DICT['Affiliate']['fields']['AffiliateDesignationID']+'_raw'][0]['id'])
            output.update({'AffiliateDesignationID':val})
            
            val = client.get_connection('AffiliateGeoReach','AffiliateGeoReachId',record[client.APP_DICT['Affiliate']['fields']['AffiliateGeoReachID']+'_raw'][0]['id'])
            output.update({'AffiliateGeoReachID':val})
            
            output.update({'updatedat':str(mdate)})
            output.update({'updatedby':str(muser)})

            affiliatename = record[client.APP_DICT['Affiliate']['fields']['AffiliateName']]
            output.update({'affiliatename':str(affiliatename)})
            
            lstateab = ''
            loc_rec = record[client.APP_DICT['Affiliate']['fields']['LocationStateAbr']+'_raw']
            if len(loc_rec) == 1:
                lstateab = loc_rec[0]['identifier']
            output.update({'locationstateabr':str(lstateab)})
            
            output_df.append(output)

    return pd.DataFrame(output_df, dtype=str)

def affiliatepercapita_runner():
    output_df = []
    records = client.find_records_updated_at_date('AffiliatePerCapita', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in apc_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['AffiliatePerCapita']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['AffiliatePerCapita']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            output.update({'UpdatedAt':str(mdate)})
            output.update({'UpdatedBy':str(muser)})
            output_df.append(output)

    return pd.DataFrame(output_df, dtype=str)

def affiliatedivision_runner():
    output_df = []
    records = client.find_records_updated_at_date('AffiliateDivision', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in adiv_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['AffiliateDivision']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['AffiliateDivision']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            val = client.get_connection('Affiliate','affiliateid',record[client.APP_DICT['AffiliateDivision']['fields']['Affiliate']+'_raw'][0]['id'])
            output.update({'Affiliate':val})

            val = client.get_connection('Division','DivisionId',record[client.APP_DICT['AffiliateDivision']['fields']['Division']+'_raw'][0]['id'])
            output.update({'Division':val})
            
            output.update({'UpdatedAt':str(mdate)})
            output.update({'UpdatedBy':str(muser)})
            out_sort = {}
            for k,v in output.items():
                if k == 'Affiliate':
                    out_sort.update({'AffiliateID':v})
                elif k == 'Division':
                    out_sort.update({'DivisionID':v})
                else:
                    out_sort.update({k:v})
            if out_sort['Action'] == 'Delete':
                if out_sort['AffiliateDivisionId']:
                    output_df.append(out_sort)
            else:
                output_df.append(out_sort)
       
    return pd.DataFrame(output_df, dtype=str)
    
def main(entity):
    if entity == 'aff':
        table_suffix = 'SYS_ARTS_KnackAffiliate_XN'
        history_suffix = 'SYS_ARTS_AffiliateUpdateHistory_XN'
        output_records = affiliate_runner()
    elif entity == 'apc':
        table_suffix = 'SYS_ARTS_KnackAffiliatePerCapita_XN'
        history_suffix = 'SYS_ARTS_AffiliatePerCapitaUpdateHistory_XN'
        output_records = affiliatepercapita_runner()
        if 'GroupNumber' in output_records:
            output_records['GroupNumber'] = output_records['GroupNumber'].astype(str).replace('\.0', '', regex=True)
    elif entity == 'adiv':
        table_suffix = 'SYS_ARTS_KnackAffiliateDivision_XN'
        history_suffix = 'SYS_ARTS_AffiliateDivisionUpdateHistory_XN'
        output_records = affiliatedivision_runner()
    
    rows = output_records.shape[0]
    columns = output_records.shape[1]

    if rows == 0 or columns == 0:
        table_name = os.environ['civis_table_path'] + table_suffix
        print('No Changes found for '+entity+'!')
        print('Truncating civis table: ' + table_name + '...')
        run = civis.io.query_civis(sql="TRUNCATE "+table_name, database='American Federation of Teachers')
        run.result()  # Wait for query to complete
        print('Done truncating!')
    else:
        print('Found '+str(rows)+' rows!')
        
        table_name = os.environ['civis_table_path'] + table_suffix
        print('Writing to main civis table: ' + table_name + '...')
        fut = civis.io.dataframe_to_civis(output_records, 'American Federation of Teachers',table_name, existing_table_rows='truncate')
        fut.result()
        print('Done writing!')

        htable_name = os.environ['civis_table_path'] + history_suffix
        print('Writing to history table: ' + htable_name + '...')
        fut = civis.io.dataframe_to_civis(output_records, 'American Federation of Teachers',htable_name, existing_table_rows='append')
        fut.result()
        print('Done writing!')

print('==== Checking Affiliate...')
main('aff')
print('==== Checking AffiliatePerCapita...')
main('apc')
print('==== Checking AffiliateDivision...')
main('adiv')
